In [1]:
#for j in $(seq 3 4); do for i in $(seq 1 20); do python main --seed $((20 * j + $i))&done;done;

In [ ]:
import bempp.api
import numpy as np

import sys
sys.path.append('../../')
from decimal import *

import bemppUQ
from bemppUQ.operators.maxwell import assemble_operators, evaluate_far_field, evaluate_far_field_sd

bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'
bempp.api.global_parameters.hmat.eps = 1E-4


from bemppUQ.shapes import reentrant_cube_exterior

def function(x, y, i, j):
    zx = 0
    zy = 0
    if i == 0:
        zx = np.sin(x * np.pi * 2)
    if i == 1:
        if x <= 0.25:
            zx = np.sin(x * np.pi * 4)
    if i == 2:
        if x > 0.25:
            zx = -np.sin(x * np.pi * 4)
    if i == 3:
        if x <= 0.5 / 3.:
            zx = np.sin(x * np.pi * 6)
    if i == 4:
        if x > 0.5 / 3. and x <= 1 / 3.:
            zx = -np.sin(x * np.pi * 6)
    if i ==5:
        if x > 1/3.:
            zx = np.sin(x * np.pi * 6)

    if j == 0:
        zy = np.sin(y * np.pi * 2)

    if j == 1:
        if y <= 0.25:
            zy = np.sin(y * np.pi * 4)

    if j == 2:
        if y > 0.25:
            zy = -np.sin(y * np.pi * 4)
    
    if j == 3:
        if y <= 0.5 / 3.:
            zy = np.sin(y * np.pi * 6)

    if j == 4:
        if y > 0.5 / 3. and y <= 1 / 3.:
            zy = -np.sin(y * np.pi * 6)
    if j == 5:
        if y > 1/3.:
            zy = np.sin(y * np.pi * 6)

    return zx * zy


# Def random:

def Phiz(point, n, domain_index, result):
    x, y, z = point

    res = 0j
    if z == 0.5 and (x <= 0.5) and (y <= 0.5):
        for ii in range(6):
            for jj in range(6):
                res += Random[ii,jj] * function(x, y, ii,jj)	
    result[0] = res



def perturbate(grid, t, kappa_pert=None):
    P1 = bempp.api.function_space(grid, 'B-P', 1)
    grid_funz = bempp.api.GridFunction(P1, fun = Phiz)
    elements = grid.leaf_view.elements
    vertices = grid.leaf_view.vertices
    normals = P1.global_dof_normals
    x, y, z = vertices
    
    vertices[2, :] = z + t * grid_funz.coefficients
    return bempp.api.grid_from_element_data(vertices, elements)



def get_base_grid(h):

    gr0 = bempp.api.shapes.reentrant_cube(h=h, refinement_factor=1) 
    elements0 = list(gr0.leaf_view.entity_iterator(0))
    N0 = len(elements0)
    tol = h / 10.
    for i in range(N0):
        el0 = elements0[i]
        z = el0.geometry.corners[2]
        if np.linalg.norm(np.array([1/2, 1/2, 1/2]) - z) < tol:
            gr0.mark(el0)
    gr1 = gr0.refine()


    elements0 = list(gr1.leaf_view.entity_iterator(0))
    N0 = len(elements0)
    for i in range(N0):
        el0 = elements0[i]
        z = el0.geometry.corners[2]
        if np.linalg.norm(np.array([1/2, 1/2, 1/2])-z) < tol:
            gr1.mark(el0)

    base_grid = gr1.refine()
    return base_grid



config = bemppUQ.config.set_case('A')
precision = 10

h = 2.0 * np.pi / (precision * config["k_int"])

base_grid = get_base_grid(h=h) 
N = 2 * base_grid.leaf_view.entity_count(1)
print(N)
transmission_operators = assemble_operators(base_grid, config)
Umean, solution = evaluate_far_field(transmission_operators, config)


#far_field_p, sol_p = evaluate_far_field_sd(base_grid, transmission_operators, config, solution, grid_fun)

nrank = 1
grid_funs = []
for ii in range(nrank):
    for jj in range(nrank):
        def fun(point, n, domain_index, result):
            x, y, z = point

            res = 0j
            if z == 0.5 and (x <= 0.5) and (y <= 0.5):
                res += function(x, y, ii,jj)
            result[0] = res
            
        space = bempp.api.function_space(base_grid, 'B-P', 1)
        grid_fun = bempp.api.GridFunction(space, fun= fun)
        grid_funs.append(grid_fun)

3204


In [ ]:
result = evaluate_far_field_sd(base_grid, transmission_operators, config, solution, grid_funs, solve = True)

In [ ]:
n = config['number_of_angles']
angles = config['angles']

Sigma = np.zeros((n, n), dtype=np.complex128)
k = 0
for ii in range(nrank):
    for jj in range(nrank):
        #rr = result[k]
        #Sigma += 1/3. * np.sum(np.kron(rr,rr).reshape(9, 400,400), axis=0).diagonal() 
        ff = np.sum(result[k], axis=0)
        Sigma += 1/3. * bemppUQ.utils.login.tensorize(ff,ff)
        k += 1
        
t = 0.05
Uvars = t ** 2 * Sigma.diagonal()
Umeans = np.sum(Umean, axis=0)


In [ ]:
from matplotlib import pyplot as plt
from bemppUQ.utils.login import tensorize

fig, ax  = plt.subplots(1,1, figsize=(6,6))

FFp = Umeans  + 2 * np.sqrt(np.abs(Uvars))
FFm = Umeans  - 2 * np.sqrt(np.abs(Uvars))

angles = config['angles']

cs = 10 * np.log10(4 * np.pi * np.abs(Umeans)**2)
cs_FOAp = 10 * np.log10(4 * np.pi * np.abs(FFp)**2)
cs_FOAm = 10 * np.log10(4 * np.pi * np.abs(FFm)**2)


ax.plot(angles, cs, '-r', label='Mean')
ax.plot(angles,cs_FOAp, '--b', label='FOAp')
ax.plot(angles,cs_FOAm, '--b', label='FOAm')

ax.set_title(r'$\mathrm{RCS}_z(\theta)$: BEM (red) \pm analytic (blue)', fontsize=14)

ax.set_xlabel(r'$\theta$ (rad)', fontsize=14)
ax.set_ylabel(r'$\mathrm{RCS}_z(\theta)$', fontsize=14)

#ax.legend()
ax.grid()

plt.show(block=False)

In [ ]:
y = np.sqrt(np.abs(Uvars))
plt.plot(angles, y)
plt.ylim(0.001,0.013)

In [ ]:
y.mean(), y.max()